In [249]:
%load_ext autoreload
%autoreload 2
import sys
print(sys.executable)
import socket
print(socket.gethostname())
import os
os.chdir("/home/icb/alioguz.can/projects/he_pathalogy")

import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/icb/alioguz.can/miniconda3/envs/he_env/bin/python
supergpu14.scidom.de


In [297]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tiff
import squidpy as sq
import seaborn  as sns
import histomicstk as htk
from collections import defaultdict
from pathlib import Path
from tqdm import tqdm
from utils import find_full_tiles, plot_top_tiles, quick_tissue_mask, macenko_stain_matrix, separate_stains_od

In [273]:
path = "/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms"

# Collect all DICOMs
all_files = sorted([p for p in os.listdir(path) if p.endswith(".dcm")])
print(f"🔍 Scanning {len(all_files)} DICOM files in: {path}\n")

# Group by prefix (before first underscore)
groups = defaultdict(list)
for f in all_files:
    prefix = f.split("_")[0]
    groups[prefix].append(f)

# Find duplicate groups
duplicates = {k: v for k, v in groups.items() if len(v) > 1}

if duplicates:
    print("⚠️ Duplicate slide prefixes found:\n")
    for prefix, files in duplicates.items():
        print(f"[{prefix}] — {len(files)} DICOMs")
        for f in files:
            full_path = os.path.join(path, f)
            try:
                ds = pydicom.dcmread(full_path, stop_before_pixels=True)
                rows, cols = ds.get("Rows", "?"), ds.get("Columns", "?")
                frames = ds.get("NumberOfFrames", "N/A")
                print(f"   {f} → {rows}×{cols} pixels, {frames} frames")
            except Exception as e:
                print(f"   {f} → [ERROR reading DICOM: {e}]")
        print()
else:
    print("✅ No duplicates by prefix found.\n")

# Summary
print("—" * 80)
print(f"📦 Total DICOM files scanned: {len(all_files)}")
print(f"📂 Duplicate prefix groups:   {len(duplicates)}")
print("—" * 80)

🔍 Scanning 48 DICOM files in: /lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms

✅ No duplicates by prefix found.

————————————————————————————————————————————————————————————————————————————————
📦 Total DICOM files scanned: 48
📂 Duplicate prefix groups:   0
————————————————————————————————————————————————————————————————————————————————


In [ ]:
### DELTES DUPLICATE FILES OF SHARED PARENT FOLDERS BY KEEPING THE FILE WITH MORE FRAMES

# path = "/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms"

# # Collect all DICOMs
# all_files = sorted([p for p in os.listdir(path) if p.endswith(".dcm")])
# print(f"🔍 Scanning {len(all_files)} DICOM files in: {path}\n")

# # Group by prefix (before first underscore)
# groups = defaultdict(list)
# for f in all_files:
#     prefix = f.split("_")[0]
#     groups[prefix].append(f)

# # Find duplicate groups
# duplicates = {k: v for k, v in groups.items() if len(v) > 1}

# deleted_count = 0
# kept_count = 0

# if duplicates:
#     print("⚠️ Duplicate slide prefixes found and processing for cleanup:\n")
#     for prefix, files in duplicates.items():
#         print(f"[{prefix}] — {len(files)} DICOMs")

#         frame_info = []
#         for f in files:
#             full_path = os.path.join(path, f)
#             try:
#                 ds = pydicom.dcmread(full_path, stop_before_pixels=True)
#                 frames = int(ds.get("NumberOfFrames", 1))
#                 frame_info.append((frames, full_path))
#                 print(f"   {f} → {frames} frames")
#             except Exception as e:
#                 print(f"   {f} → [ERROR reading DICOM: {e}]")

#         # Sort by frame count (highest last)
#         frame_info.sort(key=lambda x: x[0])
#         keep = frame_info[-1]  # highest frame count
#         to_delete = frame_info[:-1]

#         kept_count += 1
#         print(f"   ✅ Keeping: {os.path.basename(keep[1])} ({keep[0]} frames)")

#         # Delete lower-frame duplicates
#         for frames, path_to_remove in to_delete:
#             try:
#                 os.remove(path_to_remove)
#                 deleted_count += 1
#                 print(f"   ❌ Deleted: {os.path.basename(path_to_remove)} ({frames} frames)")
#             except Exception as e:
#                 print(f"   [ERROR deleting {path_to_remove}: {e}]")

#         print()
# else:
#     print("✅ No duplicates by prefix found.\n")

# # Summary
# print("—" * 80)
# print(f"📦 Total DICOM files scanned:  {len(all_files)}")
# print(f"📂 Duplicate prefix groups:    {len(duplicates)}")
# print(f"✅ Files kept (highest frames): {kept_count}")
# print(f"❌ Files deleted (lower frames): {deleted_count}")
# print("—" * 80)

In [280]:
root_path = Path("/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_dicoms")

# Get all .dcm files
all_files = sorted([p for p in os.listdir(root_path) if p.endswith(".dcm")])

# Extract prefixes (before the first underscore)
prefixes = sorted({f.split("_")[0] for f in all_files})

# Print results
print(f"📦 Total DICOM files: {len(all_files)}")
print(f"🔹 Unique prefixes: {len(prefixes)}\n")

for p in prefixes:
    print(p)

📦 Total DICOM files: 50
🔹 Unique prefixes: 50

E20012878#1#3#3
E20012878#1#4#3
E20012878#1#5#3
E21004292#2#1#3
E21006287#1#1#3
E21006287#3#1#3
E21006287#4#1#3
E21006287#5#1#3
E21006287#6#1#3
E21006288#1#1#3
E21006288#2#1#3
E21006288#3#1#3
E21006288#4#1#3
E21006288#5#1#3
E21006679#1#1#3
E21006679#2#1#3
E21006679#3#1#3
E21009820#1#1#5
E21011531#10#1#3
E21011531#11#1#3
E21011531#2#1#3
E21011531#3#1#3
E21011531#5#1#3
E21011531#6#1#3
E21011531#7#1#3
E21011531#8#1#3
E21011531#9#1#3
KO21000706#1#1#3
KO21000706#10#1#3
KO21000706#11#1#3
KO21000706#2#1#3
KO21000706#4#1#3
KO21000706#6#1#3
KO21000706#7#1#3
KO21000739#2#1#3
KO21001954#2#1#3
KO21001972#2#1#3
KO21001972#3#1#3
KO21001972#4#1#3
KO21001972#7#1#3
KO21001972#8#1#3
KO21001994#1#1#3
KO21001994#1#2#3
KO21001994#2#1#3
KO21001994#2#2#3
KO21001996#1#1#3
KO21001997#1#1#3
KO21001997#3#1#3
KO21001997#5#1#3
KO21001997#7#1#3


In [295]:
dcm_file_paths = sorted(root_path.rglob("*.dcm"))
print(len(dcm_file_paths))

50


In [ ]:
patient_dict = {}

for dicom_path in tqdm(dcm_file_paths, desc="Processing paths"):
    print(f"\nProcessing {dicom_path.name} ...")

    ds = pydicom.dcmread(dicom_path)
    image = ds.pixel_array.astype(np.float32)

    # Handle possible planar configuration
    if getattr(ds, "PlanarConfiguration", 0) == 1:
        image = np.moveaxis(image, 0, -1)  # (3, H, W) → (H, W, 3)
    elif image.shape[-1] != 3:
        image = np.transpose(image, (1, 2, 0))  # (3, H, W) → (H, W, 3)

    # Ensure (N, H, W, 3)
    if image.ndim == 3:
        image = image[None, ...]

    # --- Per-tile normalization ---
    p_low, p_high = 1, 99.5
    for i in range(image.shape[0]):
        tile = image[i]
        lo, hi = np.percentile(tile, [p_low, p_high])
        tile = np.clip((tile - lo) / (hi - lo + 1e-6), 0, 1)
        image[i] = tile * 255.0
    image = image.astype(np.uint8)

    # print("normalized shape:", image.shape, "dtype:", image.dtype)

    # Patient ID
    patient_id = ds.ContainerIdentifier
    # print(f"Patient: {patient_id}")

    # Skip if already processed
    if patient_id not in patient_dict:
        patient_dict[patient_id] = {}

    # --- Find full tiles ---
    full_idx, scores = find_full_tiles(image, SCALE_255=False)
    # print(f"{len(full_idx)} files have signal.")
    full_idx = full_idx

    # --- Compute Eosin OD for full tiles ---
    E_od_list = []
    for idx in full_idx:
        tile = image[idx]
        tmask = quick_tissue_mask(tile)
        W = macenko_stain_matrix(tile, tissue_mask=tmask, beta=0.15, alpha=1.0)
        H_od, E_od, _ = separate_stains_od(tile, W)
        E_od_list.append(E_od)
    E_od_array = np.stack(E_od_list)

    # --- Segmentation and counting ---
    for i, arr in enumerate(E_od_array):
        sq_imgCont = sq.im.ImageContainer(arr, layer="image", mask=None, dims="channels_last")
        sq.im.process(sq_imgCont, layer="image", method="smooth", sigma=1)
        threshold_val = np.percentile(np.array(sq_imgCont["image_smooth"]).flatten(), 99)
        sq.im.segment(img=sq_imgCont, layer="image_smooth", method="watershed", thresh=threshold_val, geq=True)

        seg_mask = sq_imgCont["segmented_watershed"].values
        labels = np.unique(seg_mask)
        n_objects = len(labels) - (1 if 0 in labels else 0)

        print(f"Tile {full_idx[i]}: Thresholding at {threshold_val} -> Number of objects identified: {n_objects}")
        patient_dict[patient_id][f"tile_{full_idx[i]}"] = n_objects

# --- Convert dictionary to DataFrame ---
df = pd.DataFrame.from_dict(patient_dict, orient="index").fillna(0).astype(int)
df = df.reindex(sorted(df.columns, key=lambda x: int(x.split("_")[1])), axis=1)
df["maximum"] = df.apply(
    lambda row: f"{row.idxmax()}={row.max()}", axis=1
)
df.to_csv("/home/icb/alioguz.can/projects/he_pathalogy/hne_objects_by_patient_and_tile.csv")
print(f"\nPatients: {df.shape[0]} | Tiles: {df.shape[1]}")

Process svs files

In [382]:

import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, morphology, exposure
import openslide

In [384]:
root_path = "/lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_svs"
svs_path_list = sorted(Path(root_path).rglob("*.svs"))
print(len(svs_path_list))

12


In [386]:
# Path to your SVS file
svs_path = svs_path_list[0]
print(f"Patient: {Path(svs_path).stem.split('_')[0]}")
# Open the slide
slide = openslide.OpenSlide(svs_path)
level = 1  # use level 1 for manageable resolution

# Get dimensions at this level
width, height = slide.level_dimensions[level]
tile_size = 512

# Calculate how many tiles fit across the image
n_tiles_x = width // tile_size
n_tiles_y = height // tile_size

# Preallocate array (n, h, w, c)
image = np.zeros((n_tiles_x * n_tiles_y, tile_size, tile_size, 3), dtype=np.uint8)

print(f"Slide size at level {level}: {width}x{height}")
print(f"Extracting {n_tiles_x} x {n_tiles_y} = {n_tiles_x * n_tiles_y} tiles of {tile_size}x{tile_size}px")

# Loop over grid and read tiles
idx = 0
for y in range(n_tiles_y):
    for x in range(n_tiles_x):
        # Map tile coordinates to level 0 space
        x0 = int(x * tile_size * slide.level_downsamples[level])
        y0 = int(y * tile_size * slide.level_downsamples[level])

        # Read region and convert to NumPy RGB
        region = slide.read_region((x0, y0), level, (tile_size, tile_size))
        region = np.array(region)[:, :, :3]  # drop alpha
        image[idx] = region
        idx += 1
slide.close()

full_idx, scores = find_full_tiles(image, SCALE_255=False)
print(f"{len(full_idx)} files have signal.")
full_idx = full_idx

# --- Parameters ---
plot_results = False   # set to False to disable visualization

for i in full_idx:
    # Extract blue channel only
    blue_tile = image[i, :, :, 2]  # shape (H, W)

    # Convert to optical density (OD)
    od = -np.log((blue_tile.astype(np.float32) + 1) / 255.0)

    # 1) Slight Gaussian smoothing
    od_s = filters.gaussian(od, sigma=1.0, preserve_range=True)

    # 2) Remove large-scale background (white tophat)
    od_corr = morphology.white_tophat(od_s, footprint=morphology.disk(15))

    # 3) Contrast normalize and threshold (Otsu)
    lo, hi = np.percentile(od_corr, (2, 99))
    lo = float(lo); hi = float(hi)
    if hi <= lo:
        hi = lo + 1e-6
    od_corr = exposure.rescale_intensity(od_corr, in_range=(lo, hi))
    thr = filters.threshold_otsu(od_corr)
    mask = od_corr > thr  # high OD = nuclei

    # 4) Postprocess mask
    mask = morphology.remove_small_objects(mask, min_size=20)
    mask = morphology.remove_small_holes(mask, area_threshold=20)

    # 5) Label connected components
    labeled = morphology.label(mask)
    n_objects = labeled.max()
    print(f"Tile {i}: Threshold {thr:.2f} → {n_objects} objects")

    # --- Optional visualization ---
    if plot_results:
        fig, axes = plt.subplots(1, 3, figsize=(14, 5))
        axes[0].imshow(blue_tile, cmap="gray")
        axes[0].set_title(f"Tile {i} - Blue channel")
        axes[0].axis("off")

        axes[1].imshow(od_corr, cmap="gray")
        axes[1].set_title("OD corrected (contrast-normalized)")
        axes[1].axis("off")

        axes[2].imshow(mask, cmap="gray")
        axes[2].set_title(f"Binary mask (thr={thr:.2f})")
        axes[2].axis("off")

        plt.tight_layout()
        plt.show()

Patient: KO21000758#1#1#5
Slide size at level 1: 12042x8270
Extracting 23 x 16 = 368 tiles of 512x512px
Tiles total: 368
Full tiles: 16
16 files have signal.
Tile 258: Threshold 0.38 → 183 objects
Tile 213: Threshold 0.38 → 187 objects
Tile 214: Threshold 0.38 → 150 objects
Tile 257: Threshold 0.39 → 148 objects
Tile 259: Threshold 0.38 → 156 objects
Tile 212: Threshold 0.41 → 129 objects
Tile 234: Threshold 0.42 → 113 objects
Tile 215: Threshold 0.41 → 128 objects
Tile 260: Threshold 0.40 → 125 objects
Tile 238: Threshold 0.42 → 103 objects
Tile 281: Threshold 0.42 → 83 objects
Tile 191: Threshold 0.43 → 82 objects
Tile 235: Threshold 0.41 → 106 objects
Tile 192: Threshold 0.43 → 68 objects
Tile 282: Threshold 0.44 → 73 objects
Tile 190: Threshold 0.41 → 67 objects


In [390]:
patient_dict = {}

for svs_path in svs_path_list:
    print(f"\nProcessing: {svs_path}")
    patient_id = Path(svs_path).stem.split("_")[0]
    print(f"Patient: {patient_id}")

    # --- open SVS slide ---
    slide = openslide.OpenSlide(svs_path)
    level = 1  # manageable resolution
    width, height = slide.level_dimensions[level]
    tile_size = 512

    n_tiles_x = width // tile_size
    n_tiles_y = height // tile_size
    image = np.zeros((n_tiles_x * n_tiles_y, tile_size, tile_size, 3), dtype=np.uint8)

    print(f"Slide size at level {level}: {width}x{height}")
    print(f"Extracting {n_tiles_x} x {n_tiles_y} = {n_tiles_x * n_tiles_y} tiles of {tile_size}x{tile_size}px")

    idx = 0
    for y in range(n_tiles_y):
        for x in range(n_tiles_x):
            x0 = int(x * tile_size * slide.level_downsamples[level])
            y0 = int(y * tile_size * slide.level_downsamples[level])
            region = slide.read_region((x0, y0), level, (tile_size, tile_size))
            region = np.array(region)[:, :, :3]
            image[idx] = region
            idx += 1
    slide.close()

    # --- tissue filtering ---
    full_idx, scores = find_full_tiles(image, SCALE_255=False)
    print(f"{len(full_idx)} tiles have signal.")

    if patient_id not in patient_dict:
        patient_dict[patient_id] = {}

    # --- nuclei detection ---
    plot_results = False
    for i in full_idx:
        blue_tile = image[i, :, :, 2]

        # Convert to optical density (OD)
        od = -np.log((blue_tile.astype(np.float32) + 1) / 255.0)
        od_s = filters.gaussian(od, sigma=1.0, preserve_range=True)
        od_corr = morphology.white_tophat(od_s, footprint=morphology.disk(15))

        lo, hi = np.percentile(od_corr, (2, 99))
        lo, hi = float(lo), float(hi)
        if hi <= lo:
            hi = lo + 1e-6
        od_corr = exposure.rescale_intensity(od_corr, in_range=(lo, hi))

        thr = filters.threshold_otsu(od_corr)
        mask = od_corr > thr
        mask = morphology.remove_small_objects(mask, min_size=20)
        mask = morphology.remove_small_holes(mask, area_threshold=20)

        labeled = morphology.label(mask)
        n_objects = labeled.max()

        patient_dict[patient_id][f"tile_{i}"] = n_objects
        print(f"Tile {i}: Threshold {thr:.2f} → {n_objects} objects")

        if plot_results:
            fig, axes = plt.subplots(1, 3, figsize=(14, 5))
            axes[0].imshow(blue_tile, cmap="gray")
            axes[0].set_title(f"Tile {i} - Blue channel"); axes[0].axis("off")
            axes[1].imshow(od_corr, cmap="gray")
            axes[1].set_title("OD corrected (contrast-normalized)"); axes[1].axis("off")
            axes[2].imshow(mask, cmap="gray")
            axes[2].set_title(f"Binary mask (thr={thr:.2f})"); axes[2].axis("off")
            plt.tight_layout(); plt.show()

# --- Convert dictionary to DataFrame ---
df = pd.DataFrame.from_dict(patient_dict, orient="index").fillna(0).astype(int)
df = df.reindex(sorted(df.columns, key=lambda x: int(x.split("_")[1])), axis=1)
df["maximum"] = df.apply(
    lambda row: f"{row.idxmax()}={row.max()}", axis=1
)
df.to_csv("/home/icb/alioguz.can/projects/he_pathalogy/hne_objects_by_patient_and_tile_svs.csv")
print(f"\nPatients: {df.shape[0]} | Tiles: {df.shape[1]}")


Processing: /lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_svs/KO21000758#1#1#5_e3a796a0-7913-4498-f697-0ff222d00f5f_213926.svs
Patient: KO21000758#1#1#5
Slide size at level 1: 12042x8270
Extracting 23 x 16 = 368 tiles of 512x512px
Tiles total: 368
Full tiles: 16
16 tiles have signal.
Tile 258: Threshold 0.38 → 183 objects
Tile 213: Threshold 0.38 → 187 objects
Tile 214: Threshold 0.38 → 150 objects
Tile 257: Threshold 0.39 → 148 objects
Tile 259: Threshold 0.38 → 156 objects
Tile 212: Threshold 0.41 → 129 objects
Tile 234: Threshold 0.42 → 113 objects
Tile 215: Threshold 0.41 → 128 objects
Tile 260: Threshold 0.40 → 125 objects
Tile 238: Threshold 0.42 → 103 objects
Tile 281: Threshold 0.42 → 83 objects
Tile 191: Threshold 0.43 → 82 objects
Tile 235: Threshold 0.41 → 106 objects
Tile 192: Threshold 0.43 → 68 objects
Tile 282: Threshold 0.44 → 73 objects
Tile 190: Threshold 0.41 → 67 objects

Processing: /lustre/groups/ml01/workspace/alioguz.can/hne_cg/filtered_svs/KO210007